In [1]:
import random
import pandas as pd
from datetime import datetime

# Temporary workflow
# (1) Download csv with `bootpython && heroku run --app guarded-everglades-89687 make exportdb && make importdb`
# (2) save links to ~/Documents/links.txt in onetab format
# (3) run this script (or otherwise modify the CSV as desired)
# (4) check date cell
# (5) Upload back up with `make csv_to_s3 && heroku run --app guarded-everglades-89687 make importdb && make pipeline`
# (6) Remember to delete contents of ~/Documents/links.txt
links = pd.read_csv('data/export.csv')
links

/var/folders/7f/6ty6wfcd1hs3d1wz9b1c4gsw0000gn/T/ipykernel_9421/1024356683.py:12: DtypeWarning: Columns (12) have mixed types. Specify dtype option on import or set low_memory=False.
  links = pd.read_csv('data/export.csv')


,id,url,title,summary,domain,added,modified,liked,category,aggregator,seed,tweet,starred
0,0,http://moralitylab.bc.edu/wp-content/uploads/2...,Moral Realism As Moral Motivation: The Impact ...,"""People disagree about whether 'moral facts' a...",moralitylab.bc.edu,2016-12-06 05:00:00+00,2016-12-06 05:00:00+00,1.0,ethics,custom,17.0,0.0,0
1,1,http://www.jefftk.com/news/2013-04-01,The Unintuitive Power Laws of Giving,"""Why give globally? Why give money? Why health...",www.jefftk.com,2016-12-07 05:00:00+00,2016-12-07 05:00:00+00,1.0,giving,custom,59.0,0.0,0
2,2,http://calnewport.com/blog/2013/03/24/how-to-w...,How to Write Six Important Papers a Year witho...,"""I'm fascinated by people who produce a large ...",calnewport.com,2016-12-08 05:00:00+00,2016-12-08 05:00:00+00,1.0,productivity,custom,56.0,0.0,0
3,3,http://www.freakonomics.com/2012/10/31/an-alte...,An Alternative to Democracy,"""In Glen's voting mechanism\, every voter can ...",www.freakonomics.com,2016-12-09 05:00:00+00,2016-12-09 05:00:00+00,1.0,politicalscience,custom,29.0,0.0,0
4,4,http://www.wired.com/vanish/2009/11/ff_vanish2...,Writer Evan Ratliff Tried to Vanish: Here's Wh...,"""August 13\, 6:40 PM: I'm driving East out of ...",www.wired.com,2016-12-10 05:00:00+00,2016-12-10 05:00:00+00,1.0,technology,custom,92.0,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
109426,174201,https://thezvi.substack.com/p/medical-roundup-1,Medical Roundup #1,\N,\N,2024-02-14 13:29:11.417423+00,2024-02-14 13:29:11.417423+00,NaN,\N,Zvi,56.0,0.0,\N
109427,174202,https://thezvi.substack.com/p/ai-48-the-talk-o...,AI #48: The Talk of Davos,\N,\N,2024-02-14 13:29:11.418681+00,2024-02-14 13:29:11.418681+00,NaN,\N,Zvi,41.0,0.0,\N
109428,174203,https://www.slowboring.com/p/tuesday-thread-a9b,Tuesday thread,\N,www.slowboring.com,2024-02-14 13:29:11.419931+00,2024-02-14 13:29:38.6144+00,0.0,\N,Yglesias,77.0,0.0,\N
109429,174204,https://thezvi.substack.com/p/on-the-proposed-...,On the Proposed California SB 1047,\N,\N,2024-02-14 13:29:11.421161+00,2024-02-14 13:29:11.421161+00,NaN,\N,Zvi,97.0,0.0,\N


In [2]:
links['added'].max()

'2024-02-14 14:27:52.975494+00'

In [3]:
links[(links['aggregator'] == 'Custom') & (links['liked'] != '1') & (links['liked'] != '0') & (links['liked'] != '-1')]['id']

1299        1306
1300        1307
1301        1308
1302        1309
1303        1310
           ...  
109398    174172
109399    174173
109400    174175
109401    174176
109402    174177
Name: id, Length: 16301, dtype: int64

In [4]:
links[links['url'].duplicated()]

,id,url,title,summary,domain,added,modified,liked,category,aggregator,seed,tweet,starred
109375,174149,https://joecarlsmith.com/2024/01/09/does-ai-ri...,Does AI risk “other” the AIs? - Joe Carlsmith,\N,\N,2024-02-14 14:27:52.80452+00,2024-02-14 14:27:52.804527+00,NaN,\N,Custom,95.0,0.0,1


In [5]:
links[links['url'].duplicated()]['id']

109375    174149
Name: id, dtype: int64

In [6]:
link_file = open('/Users/peterhurford/Documents/links.txt', 'r')
new_links = link_file.readlines()

In [7]:
def new_entry(links, url, title):
    return {'id': links['id'].max() + 1,
            'url': str(url).strip().replace(',', ''),
            'title': str(title).strip().replace(',', ''),
            'summary': '\\N',
            'domain': '\\N',
            'added': str(datetime.now()),
            'modified': str(datetime.now()),
            'liked': '\\N',
            'category': '\\N',
            'aggregator': 'Custom',
            'seed': random.randint(1, 100),
            'starred': '\\N',
            'tweet': 0}

for i, new_link in enumerate(new_links):
    if i % 10 == 0 or i == len(new_links) - 1:
        print('...{}/{}'.format(i + 1, len(new_links)))
    if ' | ' in new_link:
        new_link = new_link.replace('    - ', '').replace('\n', '')
        entry = new_entry(links, new_link.split(' | ')[0], new_link.split(' | ')[1])
        links = pd.concat([links, pd.DataFrame([entry])], ignore_index=True)

links

...1/25
...11/25
...21/25
...25/25


,id,url,title,summary,domain,added,modified,liked,category,aggregator,seed,tweet,starred
0,0,http://moralitylab.bc.edu/wp-content/uploads/2...,Moral Realism As Moral Motivation: The Impact ...,"""People disagree about whether 'moral facts' a...",moralitylab.bc.edu,2016-12-06 05:00:00+00,2016-12-06 05:00:00+00,1.0,ethics,custom,17.0,0.0,0
1,1,http://www.jefftk.com/news/2013-04-01,The Unintuitive Power Laws of Giving,"""Why give globally? Why give money? Why health...",www.jefftk.com,2016-12-07 05:00:00+00,2016-12-07 05:00:00+00,1.0,giving,custom,59.0,0.0,0
2,2,http://calnewport.com/blog/2013/03/24/how-to-w...,How to Write Six Important Papers a Year witho...,"""I'm fascinated by people who produce a large ...",calnewport.com,2016-12-08 05:00:00+00,2016-12-08 05:00:00+00,1.0,productivity,custom,56.0,0.0,0
3,3,http://www.freakonomics.com/2012/10/31/an-alte...,An Alternative to Democracy,"""In Glen's voting mechanism\, every voter can ...",www.freakonomics.com,2016-12-09 05:00:00+00,2016-12-09 05:00:00+00,1.0,politicalscience,custom,29.0,0.0,0
4,4,http://www.wired.com/vanish/2009/11/ff_vanish2...,Writer Evan Ratliff Tried to Vanish: Here's Wh...,"""August 13\, 6:40 PM: I'm driving East out of ...",www.wired.com,2016-12-10 05:00:00+00,2016-12-10 05:00:00+00,1.0,technology,custom,92.0,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
109451,174226,https://static1.squarespace.com/static/64edf8e...,Introduction to AI chip making in China [final],\N,\N,2024-02-14 15:15:38.191865,2024-02-14 15:15:38.191872,\N,\N,Custom,60.0,0.0,\N
109452,174227,https://futureoflife.org/wp-content/uploads/20...,FLI_Policymaking_In_The_Pause.pdf,\N,\N,2024-02-14 15:15:38.199786,2024-02-14 15:15:38.199803,\N,\N,Custom,85.0,0.0,\N
109453,174228,https://arxiv.org/pdf/2310.09217.pdf,2310.09217.pdf,\N,\N,2024-02-14 15:15:38.207280,2024-02-14 15:15:38.207289,\N,\N,Custom,68.0,0.0,\N
109454,174229,https://docs.google.com/document/d/1rPE4GDRPGa...,An exploration of emergency brakes [TAIGA] - G...,\N,\N,2024-02-14 15:15:38.215209,2024-02-14 15:15:38.215217,\N,\N,Custom,54.0,0.0,\N


In [8]:
links[(links['aggregator'] == 'Custom') & (links['liked'] != '1') & (links['liked'] != '0') & (links['liked'] != '-1')]

,id,url,title,summary,domain,added,modified,liked,category,aggregator,seed,tweet,starred
1299,1306,http://blog.beeminder.com/trust/,Trusting Your Divided Self | Beeminder Blog,\N,blog.beeminder.com,2020-01-01 05:00:00+00,2020-01-01 05:00:00+00,0.0,productivity,Custom,10.0,0.0,0
1300,1307,http://powlyglot.com/fi3m-review-part-1-focus/,"""What I Learned from Fluent in 3 Months\, Part...",\N,powlyglot.com,2020-01-01 05:00:00+00,2020-01-01 05:00:00+00,0.0,productivity,Custom,9.0,0.0,0
1301,1308,http://effective-altruism.com/ea/u8/ea_intervi...,"""EA Interview Series\, February 2016: John Sal...",\N,effective-altruism.com,2020-01-01 05:00:00+00,2020-01-01 05:00:00+00,0.0,effectivealtruism,Custom,85.0,0.0,0
1302,1309,https://fivethirtyeight.com/features/most-2020...,Most 2020 Candidates Have Something In Common:...,\N,fivethirtyeight.com,2020-01-01 05:00:00+00,2020-01-01 05:00:00+00,1.0,politicalscience,Custom,82.0,0.0,0
1303,1310,http://www.smartpassiveincome.com/backlinking-...,THE Backlinking Strategy That Works – 2014 and...,\N,www.smartpassiveincome.com,2020-01-01 05:00:00+00,2020-01-01 05:00:00+00,0.0,entrepreneurship,Custom,59.0,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
109451,174226,https://static1.squarespace.com/static/64edf8e...,Introduction to AI chip making in China [final],\N,\N,2024-02-14 15:15:38.191865,2024-02-14 15:15:38.191872,\N,\N,Custom,60.0,0.0,\N
109452,174227,https://futureoflife.org/wp-content/uploads/20...,FLI_Policymaking_In_The_Pause.pdf,\N,\N,2024-02-14 15:15:38.199786,2024-02-14 15:15:38.199803,\N,\N,Custom,85.0,0.0,\N
109453,174228,https://arxiv.org/pdf/2310.09217.pdf,2310.09217.pdf,\N,\N,2024-02-14 15:15:38.207280,2024-02-14 15:15:38.207289,\N,\N,Custom,68.0,0.0,\N
109454,174229,https://docs.google.com/document/d/1rPE4GDRPGa...,An exploration of emergency brakes [TAIGA] - G...,\N,\N,2024-02-14 15:15:38.215209,2024-02-14 15:15:38.215217,\N,\N,Custom,54.0,0.0,\N


In [9]:
def clean(txt):
    if not isinstance(txt, str):
        return txt
    txt = txt.replace('&nbsp;', '')
    txt = txt.replace('&ldquo;', '"')
    txt = txt.replace('&rdquo;', '"')
    txt = txt.replace('&lsquo;', '\'')
    txt = txt.replace('&rsquo;', '\'')
    txt = txt.replace('&mdash;', '-')
    txt = txt.replace('&ndash;', '-')
    for i in range(10):
        txt = txt.replace('"""', '"')
        txt = txt.replace('\'\'\'', '\'')
    txt = txt.replace('n"t', 'n\'t')
    txt = txt.replace('n""t', 'n\'t')
    return txt

links = links[links['id'].notnull()]   # Drop empty column
links['id'] = links['id'].astype(int)  # Fix float ID issue
links['seed'] = links['seed'].astype(int)
links['tweet'] = links['tweet'].apply(lambda x: 0 if str(x) == '\\N' else str(x).split('.')[0]).astype(int)
links['summary'] = links['summary'].apply(clean)
links['title'] = links['title'].apply(clean)
links = links.sort_values('id')
links

,id,url,title,summary,domain,added,modified,liked,category,aggregator,seed,tweet,starred
0,0,http://moralitylab.bc.edu/wp-content/uploads/2...,Moral Realism As Moral Motivation: The Impact ...,"""People disagree about whether 'moral facts' a...",moralitylab.bc.edu,2016-12-06 05:00:00+00,2016-12-06 05:00:00+00,1.0,ethics,custom,17,0,0
1,1,http://www.jefftk.com/news/2013-04-01,The Unintuitive Power Laws of Giving,"""Why give globally? Why give money? Why health...",www.jefftk.com,2016-12-07 05:00:00+00,2016-12-07 05:00:00+00,1.0,giving,custom,59,0,0
2,2,http://calnewport.com/blog/2013/03/24/how-to-w...,How to Write Six Important Papers a Year witho...,"""I'm fascinated by people who produce a large ...",calnewport.com,2016-12-08 05:00:00+00,2016-12-08 05:00:00+00,1.0,productivity,custom,56,0,0
3,3,http://www.freakonomics.com/2012/10/31/an-alte...,An Alternative to Democracy,"""In Glen's voting mechanism\, every voter can ...",www.freakonomics.com,2016-12-09 05:00:00+00,2016-12-09 05:00:00+00,1.0,politicalscience,custom,29,0,0
4,4,http://www.wired.com/vanish/2009/11/ff_vanish2...,Writer Evan Ratliff Tried to Vanish: Here's Wh...,"""August 13\, 6:40 PM: I'm driving East out of ...",www.wired.com,2016-12-10 05:00:00+00,2016-12-10 05:00:00+00,1.0,technology,custom,92,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
109451,174226,https://static1.squarespace.com/static/64edf8e...,Introduction to AI chip making in China [final],\N,\N,2024-02-14 15:15:38.191865,2024-02-14 15:15:38.191872,\N,\N,Custom,60,0,\N
109452,174227,https://futureoflife.org/wp-content/uploads/20...,FLI_Policymaking_In_The_Pause.pdf,\N,\N,2024-02-14 15:15:38.199786,2024-02-14 15:15:38.199803,\N,\N,Custom,85,0,\N
109453,174228,https://arxiv.org/pdf/2310.09217.pdf,2310.09217.pdf,\N,\N,2024-02-14 15:15:38.207280,2024-02-14 15:15:38.207289,\N,\N,Custom,68,0,\N
109454,174229,https://docs.google.com/document/d/1rPE4GDRPGa...,An exploration of emergency brakes [TAIGA] - G...,\N,\N,2024-02-14 15:15:38.215209,2024-02-14 15:15:38.215217,\N,\N,Custom,54,0,\N


In [10]:
links.to_csv('data/export.csv', index=False)